# Cleaning the data and providing a small and easily parsable database.

The database with the full logs is too huge to work on. Most of the data would only be needed for debugging but not for evaluation.
Thus, we create a clean and small database for the further evaluation.

In [15]:
import pandas as pd
from _conf import EXTENDED_BASELINE_DATA, PREPROCESSED_RESULTS, RESULT_FOLDER
from algbench import read_as_pandas

In [16]:
# Read the baseline results for comparison. They also already contain instance data (num_clauses and num_features).
baseline_data = pd.read_json(EXTENDED_BASELINE_DATA)
baseline_data.rename(columns={"sample_size": "baseline_sample_size"}, inplace=True)
baseline_data

,instance_name,num_vars,num_clauses,baseline_alg,baseline_alg_conf,runtime,baseline_sample_size,path_to_baseline_sample,archive_path
0,calculate,9,15,ACTS-IPOG-FT,t2,1620,9.0,QuBRA/FeatJAR/evaluation-sampling-algorithms/r...,./EXTERNAL_INPUT/baseline.zip
1,lcm,9,16,ACTS-IPOG-FT,t2,485,9.0,QuBRA/FeatJAR/evaluation-sampling-algorithms/r...,./EXTERNAL_INPUT/baseline.zip
2,email,10,17,ACTS-IPOG-FT,t2,514,8.0,QuBRA/FeatJAR/evaluation-sampling-algorithms/r...,./EXTERNAL_INPUT/baseline.zip
3,ChatClient,14,20,ACTS-IPOG-FT,t2,538,10.0,QuBRA/FeatJAR/evaluation-sampling-algorithms/r...,./EXTERNAL_INPUT/baseline.zip
4,toybox_2006-10-31_23-30-06,16,13,ACTS-IPOG-FT,t2,485,13.0,QuBRA/FeatJAR/evaluation-sampling-algorithms/r...,./EXTERNAL_INPUT/baseline.zip
...,...,...,...,...,...,...,...,...,...
1945,freetz,31012,102705,FIDE-YASA,t2_m10_null,900009,NaN,None,./EXTERNAL_INPUT/900_seconds_5_it.zip
1946,freetz,31012,102705,FIDE-YASA,t2_m10_null,900008,NaN,None,./EXTERNAL_INPUT/900_seconds_5_it.zip
1947,freetz,31012,102705,FIDE-YASA,t2_m10_null,900008,NaN,None,./EXTERNAL_INPUT/900_seconds_5_it.zip
1948,freetz,31012,102705,FIDE-YASA,t2_m10_null,900008,NaN,None,./EXTERNAL_INPUT/900_seconds_5_it.zip


In [17]:
# Get the data from the result folder
samplns_data = read_as_pandas(
    RESULT_FOLDER,
    lambda result: {
        "parameters": result["parameters"],
        "initial_sample_path": result["parameters"]["args"]["initial_sample_path"],
        "instance_name": result["parameters"]["args"]["instance_name"],
        "lower_bound": result["result"]["lower_bound"],
        "upper_bound": len(result["result"]["solution"]),
        "iteration_info": result["result"]["iteration_info"],
        # "time_used_by_yasa": result["result"]["time_used_by_yasa"],
        # "timelimit_for_samplns": result["result"]["timelimit_for_samplns"],
        # "samplns_used": result["result"]["samplns_used"],
        "runtime": result["runtime"],
        "time_limit": result["parameters"]["args"]["time_limit"],
    }
    if result.get("result", None) and result.get("result", {}).get("solution", None)
    else None,
)
samplns_data.drop_duplicates(subset=["initial_sample_path"], inplace=True)
samplns_data

,parameters,initial_sample_path,instance_name,lower_bound,upper_bound,iteration_info,runtime,time_limit
0,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/34_1_2_5_sample.csv,WaterlooGenerated,82.0,82,"[{'nbrhd_tuples': 244, 'nbrhd_confs': 57, 'ite...",653.266547,900
1,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/2_1_2_4_sample.csv,lcm,6.0,6,"[{'nbrhd_tuples': 37, 'nbrhd_confs': 9, 'itera...",0.090355,900
2,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/29_1_2_1_sample.csv,toybox_2020-12-06_00-02-46,7.0,14,"[{'nbrhd_tuples': 163, 'nbrhd_confs': 8, 'iter...",932.168702,900
3,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/32_1_2_5_sample.csv,busybox_2007-01-24_09-14-09,21.0,22,"[{'nbrhd_tuples': 225, 'nbrhd_confs': 17, 'ite...",904.828527,900
4,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/15_1_2_4_sample.csv,dell,31.0,31,"[{'nbrhd_tuples': 247, 'nbrhd_confs': 32, 'ite...",90.828721,900
...,...,...,...,...,...,...,...,...
725,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/2_1_2_1_sample.csv,lcm,6.0,6,"[{'nbrhd_tuples': 37, 'nbrhd_confs': 9, 'itera...",0.056590,900
726,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/15_1_2_3_sample.csv,dell,31.0,31,"[{'nbrhd_tuples': 240, 'nbrhd_confs': 32, 'ite...",76.992205,900
727,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/11_1_2_4_sample.csv,TightVNC,8.0,8,"[{'nbrhd_tuples': 218, 'nbrhd_confs': 10, 'ite...",74.468470,900
728,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/47_1_2_4_sample.csv,ea2468,31.0,47,"[{'nbrhd_tuples': 221, 'nbrhd_confs': 9, 'iter...",916.822741,900


In [18]:
# Pull the baseline info into the data frame
samplns_data_with_baseline_infos = baseline_data.dropna(
    subset="path_to_baseline_sample"
).merge(
    right=samplns_data,
    left_on="path_to_baseline_sample",
    right_on="initial_sample_path",
    # how="right",  # The data may still contain YASA-samples that are corrupted. Not doing `how=right` will drop them.
    suffixes=("_baseline", "_samplns"),
)
samplns_data_with_baseline_infos.drop(
    columns=["instance_name_baseline", "initial_sample_path"], inplace=True
)
samplns_data_with_baseline_infos.rename(
    columns={"instance_name_samplns": "instance_name"}, inplace=True
)
samplns_data_with_baseline_infos

,num_vars,num_clauses,baseline_alg,baseline_alg_conf,runtime_baseline,baseline_sample_size,path_to_baseline_sample,archive_path,parameters,instance_name,lower_bound,upper_bound,iteration_info,runtime_samplns,time_limit
0,9,16,Incling,t2,165,9.0,2023-03-01_13-51-03/2_1_2_1_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",lcm,6.0,6,"[{'nbrhd_tuples': 37, 'nbrhd_confs': 9, 'itera...",0.056590,900
1,9,16,Incling,t2,164,9.0,2023-03-01_13-51-03/2_1_2_4_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",lcm,6.0,6,"[{'nbrhd_tuples': 37, 'nbrhd_confs': 9, 'itera...",0.090355,900
2,10,17,Incling,t2,162,8.0,2023-03-01_13-51-03/3_1_2_3_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",email,6.0,6,"[{'nbrhd_tuples': 70, 'nbrhd_confs': 8, 'itera...",0.087525,900
3,14,20,Incling,t2,144,10.0,2023-03-01_13-51-03/4_1_2_4_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",ChatClient,7.0,7,"[{'nbrhd_tuples': 176, 'nbrhd_confs': 10, 'ite...",9.369164,900
4,16,13,Incling,t2,175,12.0,2023-03-01_13-51-03/5_1_2_1_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",toybox_2006-10-31_23-30-06,8.0,8,"[{'nbrhd_tuples': 238, 'nbrhd_confs': 10, 'ite...",9.872016,900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,1408,3319,FIDE-YASA,t2_m10_null,27439,67.0,2023-03-01_13-51-03/47_1_6_1_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",ea2468,22.0,46,"[{'nbrhd_tuples': 98, 'nbrhd_confs': 1, 'itera...",904.549790,900
457,1408,3319,FIDE-YASA,t2_m10_null,27323,67.0,2023-03-01_13-51-03/47_1_6_2_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",ea2468,23.0,44,"[{'nbrhd_tuples': 54, 'nbrhd_confs': 3, 'itera...",957.226290,900
458,1408,3319,FIDE-YASA,t2_m10_null,27180,67.0,2023-03-01_13-51-03/47_1_6_3_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",ea2468,30.0,46,"[{'nbrhd_tuples': 226, 'nbrhd_confs': 3, 'iter...",907.098987,900
459,1408,3319,FIDE-YASA,t2_m10_null,27262,67.0,2023-03-01_13-51-03/47_1_6_4_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",ea2468,24.0,48,"[{'nbrhd_tuples': 235, 'nbrhd_confs': 7, 'iter...",918.218274,900


In [22]:
# shows first 100 lines of pandas
pd.set_option("display.max_rows", 114)
# Check that we have all results.
samplns_data_with_baseline_infos.groupby(
    ["instance_name", "baseline_alg", "baseline_alg_conf"]
).size()

instance_name                baseline_alg  baseline_alg_conf
APL                          FIDE-YASA     t2_m10_null          3
                                           t2_m1_null           5
                             Incling       t2                   2
APL-Model                    FIDE-YASA     t2_m10_null          4
                                           t2_m1_null           5
                             Incling       t2                   1
BankingSoftware              FIDE-YASA     t2_m10_null          3
                                           t2_m1_null           5
BattleofTanks                FIDE-YASA     t2_m10_null          5
                                           t2_m1_null           5
ChatClient                   FIDE-YASA     t2_m10_null          5
                                           t2_m1_null           5
                             Incling       t2                   1
DMIE                         FIDE-YASA     t2_m10_null          5
               

In [20]:
# Will stop the notebook if the data is bad
# This will probably trigger because not all baseline algorithms were able to compute samples for all instances. -> Adapt as soon as the experiment finishes.
assert (
    samplns_data_with_baseline_infos.groupby(["instance_name", "baseline_alg"])
    .size()
    .min()
    == 5
)
assert (
    samplns_data_with_baseline_infos.groupby(["instance_name", "baseline_alg"])
    .size()
    .max()
    == 5
)

AssertionError: 

In [23]:
# Save the cleaned data to a json file
samplns_data_with_baseline_infos.to_json(PREPROCESSED_RESULTS)

In [ ]:
# Check that we can read the data
data = pd.read_json(PREPROCESSED_RESULTS)
data

,num_vars,num_clauses,baseline_alg,baseline_alg_conf,runtime_baseline,baseline_sample_size,path_to_baseline_sample,archive_path,parameters,instance_name,lower_bound,upper_bound,iteration_info,runtime_samplns,time_limit
0,22,40,Incling,t2,170,12,2023-03-01_13-51-03/8_1_2_1_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",FameDB,8,8,"[{'nbrhd_tuples': 224, 'nbrhd_confs': 11, 'ite...",4.386883,900
1,39,77,Incling,t2,205,22,2023-03-01_13-51-03/14_1_2_1_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",SortingLine,9,9,"[{'nbrhd_tuples': 207, 'nbrhd_confs': 16, 'ite...",37.380263,900
2,230,1181,Incling,t2,18088,406,2023-03-01_13-51-03/24_1_2_5_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",fiasco_2017-09-26_11-30-56,222,225,"[{'nbrhd_tuples': 243, 'nbrhd_confs': 83, 'ite...",923.682804,900
3,9,15,FIDE-YASA,t2_m1_null,210,9,2023-03-01_13-51-03/1_1_3_1_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",calculate,5,5,"[{'nbrhd_tuples': 38, 'nbrhd_confs': 9, 'itera...",0.081578,900
4,9,15,FIDE-YASA,t2_m1_null,173,9,2023-03-01_13-51-03/1_1_3_2_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",calculate,5,5,"[{'nbrhd_tuples': 38, 'nbrhd_confs': 9, 'itera...",0.079881,900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,1408,3319,FIDE-YASA,t2_m10_null,27439,67,2023-03-01_13-51-03/47_1_6_1_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",ea2468,22,46,"[{'nbrhd_tuples': 98, 'nbrhd_confs': 1, 'itera...",904.549790,900
407,1408,3319,FIDE-YASA,t2_m10_null,27323,67,2023-03-01_13-51-03/47_1_6_2_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",ea2468,23,44,"[{'nbrhd_tuples': 54, 'nbrhd_confs': 3, 'itera...",957.226290,900
408,1408,3319,FIDE-YASA,t2_m10_null,27180,67,2023-03-01_13-51-03/47_1_6_3_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",ea2468,30,46,"[{'nbrhd_tuples': 226, 'nbrhd_confs': 3, 'iter...",907.098987,900
409,1408,3319,FIDE-YASA,t2_m10_null,27262,67,2023-03-01_13-51-03/47_1_6_4_sample.csv,./EXTERNAL_INPUT/900_seconds_5_it.zip,"{'func': 'run_samplns', 'args': {'instance_nam...",ea2468,24,48,"[{'nbrhd_tuples': 235, 'nbrhd_confs': 7, 'iter...",918.218274,900


* `instance_name`: The instance name.
* `num_vars`: The number of variables in the instance.
* `num_clauses`: The number of clauses in the instance.
* `baseline_alg`: The baseline algorithm that created the sample
* `baseline_alg_conf`: The configuration of the baseline algorithm that created the sample.
* `runtime_baseline`: The runtime of the baseline algorithm that created the sample.
* `baseline_sample_size`: The size of the sample created by the baseline algorithm.
* `path_to_baseline_sample`: The path to the sample created by the baseline algorithm.
* `parameters` The parameters used to run samplns.
* `lower_bound` The lower bound computed by SampLNS.
* `upper_bound` The upper bound computed by SampLNS.
* `iteration_info` Info the extract the progress of SampLNS. The time do not include the baseline algorithm.
* `time_used_by_yasa` The time used by the baseline algorithm.
* `timelimit_for_samplns` The time limit for SampLNS. Check also `parameters` for the full timelimit.
* `runtime_samplns` The runtime of SampLNS without the baseline.
* `time_limit` The overall timelimit.
